In [1]:
from torch import nn
from torch.autograd import Variable
import torch
import torch.nn.functional as F
import torchvision
import torch.utils.data as data
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
from PIL import Image
import os
import matplotlib.pyplot as plt
import time
from torchsummary import summary
import config
from facenet_pytorch import training
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
from PIL import Image
import glob
import torchvision.models as models
#import models
import math

In [2]:
# Root directory for dataset
data_root = "/home/mehmetyavuz/datasets/CelebA128/"
attr_root = "/home/mehmetyavuz/datasets/list_attr_celeba.txt"
# Number of workers for dataloader
workers = 4

# Batch size during training
batch_size = 64

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = (128,128)
epochs = 100

In [3]:
class CelebA(data.Dataset):
    def __init__(self, data_path, attr_path, image_size, mode, selected_attrs):
        super(CelebA, self).__init__()
        self.data_path = data_path
        att_list = open(attr_path, 'r', encoding='utf-8').readlines()[1].split()
        atts = [att_list.index(att) + 1 for att in selected_attrs]
        images = np.loadtxt(attr_path, skiprows=2, usecols=[0], dtype=np.str)
        labels = np.loadtxt(attr_path, skiprows=2, usecols=atts, dtype=np.int)
        
        self.tf = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
            ])
        if mode == 'train':
            self.images = images[:162770]
            self.labels = labels[:162770]

        if mode == 'valid':
            self.images = images[162770:182637]
            self.labels = labels[162770:182637]

        if mode == 'test':
            self.images = images[182637:]
            self.labels = labels[182637:]
                                       
        self.length = len(self.images)
    def __getitem__(self, index):
        img = self.tf(Image.open(os.path.join(self.data_path, self.images[index])))
        att = torch.tensor((self.labels[index] + 1) // 2)
        return img, att.to(torch.float32)
    def __len__(self):
        return self.length

In [4]:
attrs_default = ["5_o_Clock_Shadow", "Arched_Eyebrows", "Attractive", "Bags_Under_Eyes", "Bald", "Bangs", "Big_Lips", "Big_Nose", 
                 "Black_Hair", "Blond_Hair", "Blurry", "Brown_Hair", "Bushy_Eyebrows", "Chubby", "Double_Chin", "Eyeglasses", "Goatee", 
                 "Gray_Hair", "Heavy_Makeup", "High_Cheekbones", "Male", "Mouth_Slightly_Open", "Mustache", "Narrow_Eyes", "No_Beard", 
                 "Oval_Face", "Pale_Skin", "Pointy_Nose", "Receding_Hairline", "Rosy_Cheeks", "Sideburns", "Smiling", "Straight_Hair", 
                 "Wavy_Hair", "Wearing_Earrings", "Wearing_Hat", "Wearing_Lipstick", "Wearing_Necklace", "Wearing_Necktie", "Young"]

In [5]:
dataset = CelebA(data_root, attr_root, image_size, 'test', attrs_default)
test_loader = torch.utils.data.DataLoader(dataset,
                                          batch_size=batch_size,
                                          shuffle=False,
                                          num_workers=workers)

In [6]:
# Decide which device we want to run on
device = torch.device("cuda:0")

In [7]:
resnet = models.__dict__['alexnet'](pretrained=False)
resnet.classifier[6] = nn.Linear(4096,40,bias=True)
resnet = torch.nn.DataParallel(resnet)
resnet.cuda()

DataParallel(
  (module): AlexNet(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
      (1): ReLU(inplace=True)
      (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (4): ReLU(inplace=True)
      (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): ReLU(inplace=True)
      (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): ReLU(inplace=True)
      (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
    (classifier): Sequential(
      (0): Dropout(p=0.5, inplace=False)
   

In [8]:
weights = torch.load("alexnet_PD_001_0_normal.pth")

In [9]:
resnet.load_state_dict(weights,strict=True)

<All keys matched successfully>

In [10]:
features = torch.FloatTensor(len(dataset), 40)
targets = torch.LongTensor(len(dataset),40)

In [11]:
ptr = 0
for i, batch in enumerate(test_loader):
    images = batch[0].cuda(non_blocking=True)
    target = batch[1].cuda(non_blocking=True)
    output = resnet(images)
    output = nn.Sigmoid()(output)
    
    b = output.size(0)
    
    features[ptr:ptr+b].copy_(output.detach())
    targets[ptr:ptr+b].copy_(target.detach())
    ptr += b

/opt/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [12]:
from sklearn.metrics import accuracy_score, roc_auc_score

In [13]:
for i, att in enumerate(attrs_default):
    print(att, ';', (targets[:,i] == torch.round(features[:,i])).float().mean().item())
print('Average Accuracy ;', (targets == torch.round(features)).float().mean().item())

5_o_Clock_Shadow ; 0.9056206941604614
Arched_Eyebrows ; 0.7786794900894165
Attractive ; 0.7748722434043884
Bags_Under_Eyes ; 0.8101392388343811
Bald ; 0.9782586693763733
Bangs ; 0.9403867125511169
Big_Lips ; 0.6846007704734802
Big_Nose ; 0.8042781352996826
Black_Hair ; 0.8489129543304443
Blond_Hair ; 0.9433423280715942
Blurry ; 0.9499047994613647
Brown_Hair ; 0.8495140671730042
Bushy_Eyebrows ; 0.8727081418037415
Chubby ; 0.9409377574920654
Double_Chin ; 0.9527602195739746
Eyeglasses ; 0.9717963933944702
Goatee ; 0.9526099562644958
Gray_Hair ; 0.9719467163085938
Heavy_Makeup ; 0.8704037666320801
High_Cheekbones ; 0.8173529505729675
Male ; 0.9354273080825806
Mouth_Slightly_Open ; 0.811491847038269
Mustache ; 0.960625171661377
Narrow_Eyes ; 0.849263608455658
No_Beard ; 0.9008616209030151
Oval_Face ; 0.7137561440467834
Pale_Skin ; 0.9578198790550232
Pointy_Nose ; 0.7153090834617615
Receding_Hairline ; 0.9156396985054016
Rosy_Cheeks ; 0.9289149641990662
Sideburns ; 0.9593226909637451
Smili